<a href="https://colab.research.google.com/github/pachterlab/CP_2023/blob/main/notebooks/celltypePred_Fig4a/gehring_nsc_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Download datasets

import requests
import os



from tqdm import tnrange, tqdm_notebook
def download_file(doi,ext):
	url = 'https://api.datacite.org/dois/'+doi+'/media'
	r = requests.get(url).json()
	netcdf_url = r['data'][0]['attributes']['url']
	r = requests.get(netcdf_url,stream=True)
	#Set file name
	fname = doi.split('/')[-1]+ext
	#Download file with progress bar
	if r.status_code == 403:
		print("File Unavailable")
	if 'content-length' not in r.headers:
		print("Did not get file")
	else:
		with open(fname, 'wb') as f:
			total_length = int(r.headers.get('content-length'))
			pbar = tnrange(int(total_length/1024), unit="B")
			for chunk in r.iter_content(chunk_size=1024):
				if chunk:
					pbar.update()
					f.write(chunk)
		return fname




#De-multiplexed h5ad with gene counts for all NSCs, labeled in each condition (from Gehring et al 2019)
download_file('10.22002/D1.1997','.gz')


os.system("gunzip *.gz")


os.system("mv D1.1997 multiplex.h5ad")



<ipython-input-1-867e2f44b0d4>:24: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  pbar = tnrange(int(total_length/1024), unit="B")


  0%|          | 0/184203 [00:00<?, ?B/s]

0

In [ ]:

!pip3 install --quiet torch
!pip3 install --quiet anndata
!pip3 install --quiet matplotlib
!pip3 install --quiet scikit-learn
!pip3 install --quiet torchsummary
!pip install --quiet scanpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.3/330.3 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.2/300.2 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.7/730.7 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 37.1 MB/s eta 

## **Install Packages**

In [ ]:
import anndata
import pandas as pd
import numpy as np


import random
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.neighbors import NeighborhoodComponentsAnalysis, NearestNeighbors
from sklearn.metrics import pairwise_distances
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import scale
import torch
import time
import scanpy as sc
import seaborn as sns
import umap
from scipy import stats
import scipy.io as sio
sns.set_style('white')

INFO:lightning_fabric.utilities.seed:Global seed set to 0


## **Import Data**

In [ ]:
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams['axes.linewidth'] = 0.1

state = 42
ndims = 2

data_path = './'

pcs = 50
n_latent = 50

In [ ]:
adata = anndata.read('multiplex.h5ad')
adata

/usr/local/lib/python3.10/dist-packages/anndata/compat/__init__.py:229: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/usr/local/lib/python3.10/dist-packages/anndata/compat/__init__.py:229: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


AnnData object with n_obs × n_vars = 21232 × 1221
    obs: 'n_genes', 'n_counts', 'LouvainRes1.5', 'LouvainRes2', 'TSNE_1', 'TSNE_2', 'assignments'
    var: 'n_cells'
    uns: 'LouvainRes1.5_colors', 'LouvainRes2_colors', 'louvain', 'neighbors', 'pca'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [ ]:
adata.obsm['X_pca'].shape

(21232, 50)

In [ ]:
tokeep = list(adata.obs['assignments'].value_counts()[adata.obs['assignments'].value_counts() > 50].index)

In [ ]:
adata = adata[adata.obs['assignments'].isin(tokeep)]
adata

View of AnnData object with n_obs × n_vars = 20568 × 1221
    obs: 'n_genes', 'n_counts', 'LouvainRes1.5', 'LouvainRes2', 'TSNE_1', 'TSNE_2', 'assignments'
    var: 'n_cells'
    uns: 'LouvainRes1.5_colors', 'LouvainRes2_colors', 'louvain', 'neighbors', 'pca'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [ ]:
def knn_infer(embd_space, labeled_idx, labeled_lab, unlabeled_idx,n_neighbors=50):
	"""
	Predicts the labels of unlabeled data in the embedded space with KNN.
	Parameters
	----------
	embd_space : ndarray (n_samples, embedding_dim)
		Each sample is described by the features in the embedded space.
		Contains all samples, both labeled and unlabeled.
	labeled_idx : list
		Indices of the labeled samples (used for training the classifier).
	labeled_lab : ndarray (n_labeled_samples)
		Labels of the labeled samples.
	unlabeled_idx : list
		Indices of the unlabeled samples.
	Returns
	-------
	pred_lab : ndarray (n_unlabeled_samples)
		Inferred labels of the unlabeled samples.
	"""

	# obtain labeled data and unlabled data from indices
	labeled_samp = embd_space[labeled_idx, :]
	unlabeled_samp = embd_space[unlabeled_idx, :]

	from sklearn.neighbors import KNeighborsClassifier

	knn = KNeighborsClassifier(n_neighbors=n_neighbors)
	knn.fit(labeled_samp, labeled_lab)

	pred_lab = knn.predict(unlabeled_samp)
	return pred_lab


## **Test Condition Prediction In Different Embedding Spaces**

In [ ]:
lab1 = list(adata.obs['assignments'])

In [ ]:
#50D PCA --> 2D
ndims = 2
acc_score_2D = []

for i in range(3):
  reducer = umap.UMAP(n_components = ndims)
  tsne = TSNE(n_components = ndims)

  x_pca = adata.obsm['X_pca']

  pcaUMAP = reducer.fit_transform(x_pca)
  pcaTSNE = tsne.fit_transform(x_pca)

  #Partially labeled UMAP

  labels = np.array([lab1]).copy().astype(np.int8)
  train_inds = np.random.choice(len(x_pca), size = int(0.7*len(x_pca)),replace=False) #0.7 for training fraction
  #Set 30% to no label (nan)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = -1

  pcaUMAPLab = reducer.fit_transform(x_pca,y=labels[0])

  preds = knn_infer(pcaUMAPLab, train_inds, adata.obs['assignments'].values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs['assignments'].values[unlab_inds], preds)
  acc_score_2D.append(acc)

  preds = knn_infer(pcaUMAP, train_inds, adata.obs['assignments'].values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs['assignments'].values[unlab_inds], preds)
  acc_score_2D.append(acc)

  preds = knn_infer(pcaTSNE, train_inds, adata.obs['assignments'].values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs['assignments'].values[unlab_inds], preds)
  acc_score_2D.append(acc)

print(acc_score_2D)

[0.3616917841516772, 0.32620320855614976, 0.3790309512234646, 0.32993031923513205, 0.31907308377896615, 0.37433155080213903, 0.3587749149246475, 0.3270134500081024, 0.38227191703127533]


PCA 50D loss

In [ ]:
# Reconstruction loss only
acc_scorePCA = []

for i in range(3):

  tsvd = TruncatedSVD(n_components=pcs)
  x_pca = tsvd.fit_transform(adata.X)

  labels = np.array([lab1])
  train_inds = np.random.choice(len(adata.X), size = int(0.7*len(adata.X)),replace=False)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = np.nan

  unlabeled_idx = []
  for i in range(len(adata)):
      if i not in train_inds:
          unlabeled_idx.append(i)

  preds = knn_infer(x_pca, train_inds, adata.obs['assignments'].values[train_inds], unlabeled_idx)
  acc = accuracy_score(adata.obs['assignments'].values[unlabeled_idx], preds)
  acc_scorePCA.append(acc)

print(acc_scorePCA)
  # print(f"nnNCA fit in {toc - tic:0.4f} seconds")

[0.44287797763733594, 0.45065629557608167, 0.4419056878949927]


PCA 100D

In [ ]:
# Reconstruction loss only
acc_scorePCA100D = []

for i in range(3):

  tsvd = TruncatedSVD(n_components=100)
  x_pca = tsvd.fit_transform(adata.X)

  labels = np.array([lab1])
  train_inds = np.random.choice(len(adata.X), size = int(0.7*len(adata.X)),replace=False)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = np.nan

  unlabeled_idx = []
  for i in range(len(adata)):
      if i not in train_inds:
          unlabeled_idx.append(i)

  preds = knn_infer(x_pca, train_inds, adata.obs['assignments'].values[train_inds], unlabeled_idx)
  acc = accuracy_score(adata.obs['assignments'].values[unlabeled_idx], preds)
  acc_scorePCA100D.append(acc)

print(acc_scorePCA100D)
  # print(f"nnNCA fit in {toc - tic:0.4f} seconds")

[0.41776049262680276, 0.41403338194782047, 0.40965807810727595]


In [ ]:
vals = pd.DataFrame()

vals['Accuracy'] = acc_score_2D + acc_scorePCA + acc_scorePCA100D
vals['Embed'] = ['PCA UMAP Sup.','PCA UMAP','PCA t-SNE']*3 + ['PCA 50D']*3 + ['PCA 100D']*3
vals['Label'] = ['Condition']*15

In [ ]:
vals.head()

,Accuracy,Embed,Label
0,0.361692,PCA UMAP Sup.,Condition
1,0.326203,PCA UMAP,Condition
2,0.379031,PCA t-SNE,Condition
3,0.329930,PCA UMAP Sup.,Condition
4,0.319073,PCA UMAP,Condition


In [ ]:
from google.colab import files

vals.to_csv('allNSCPreds.csv')
files.download('allNSCPreds.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>